#### Scrape bbcgoodfood.com recipes' unique URLs from 'index' pages

In [5]:
# ---------------------------------------------------------------------
## import modules
# ---------------------------------------------------------------------
# Python 2 & 3 Compatibility
from __future__ import print_function, division

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
import re
import time
import sys


#### Generate generic list of index page urls for scraping recipe links/urls

In [7]:
bbcgf_urlpgList = ['http://www.bbcgoodfood.com/search/recipes?query=']

R_host = 'http://www.bbcgoodfood.com/'
path = '/search?query=&page='

for p in range(1,672+1):
# for p in range(216,217+1):
    bbcgf_urlpgList.append((R_host + path + str(p)))

#### Slight tweak to by-pass website access

In [ ]:
headers = {'User-agent': 'Mozilla/5.0'}

cnt=0

for pg in bbcgf_urlpgList:
    pgResp = requests.get(pg, headers=headers)
    
    if cnt%10==0:
        print(cnt)

    if pgResp.status_code == 200:
        soup = BeautifulSoup(pgResp.content, 'lxml')
        
        tmp = soup.find("div",{"class":"view-content"}).find_all("article",{"class":"node node-recipe node-teaser-item clearfix"})

        if cnt==0:
            Links = [link.a["href"] for link in tmp]
        elif cnt>0:
            tmp2 = [link.a["href"] for link in tmp]
            Links.extend(tmp2)
        cnt += 1

In [9]:
# len(Links) #10081

#### Collect recipe links

In [133]:
R_links = pd.Series(Links, name='URL')
# R_links = R_links[0].replace(['0'],'URL')
R_links = R_links.to_frame()
R_links.to_csv('BBCgoodfood_recipesURL.csv')

In [135]:
R_links.head()

,URL
0,/recipes/4942/lemon-drizzle-cake
1,/recipes/3228/chilli-con-carne
2,/recipes/3092/ultimate-chocolate-cake
3,/recipes/1223/bestever-brownies
4,/recipes/3229/yummy-scrummy-carrot-cake


#### Now use recipe links to get recipe info.

In [9]:
# ---------------------------------------------------------------------
## import modules
# ---------------------------------------------------------------------
# Python 2 & 3 Compatibility
# from __future__ import print_function, division

# import pandas as pd
# import numpy as np

# from bs4 import BeautifulSoup
# import requests
# import re
# import time
# import sys

# credits: base recipe-scraper script with modification from https://github.com/hhursev/recipe-scraper
# avoid reinventing the wheel....
from recipe_scrapers import scrap_me 

import pickle

In [10]:
R_links = pd.read_csv('BBCgoodfood_recipesURL.csv', index_col=[0] )
# R_links

In [11]:
scrapeInt = [int(i) for i in np.round(np.linspace(0, 10081,100))]
# scrapeInt = [int(i) for i in scrapeInt]

In [12]:
R_host = 'http://www.bbcgoodfood.com/'

In [13]:
sys.setrecursionlimit(100000) # else pickle would not pickle.... =S

In [18]:
folderpath='/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/bbc_goodfood_recipes_pickles/'

In [19]:
# for sI in range(33,34): #redo
for sI in range(1,len(scrapeInt)):
    print('scrape_chunk_loop : ' + format(sI) )

    scrapedInfo =[]
    cnt = 0
    for n in R_links.URL[scrapeInt[sI-1] : scrapeInt[sI]]: #[:100]:

        if cnt%10==0:
            print(cnt)

        tmp = scrap_me(R_host+n)
        scrapedInfo.append(tmp)

        cnt+=1

    filename = (folderpath +"scrapedInfo_BBCgdfd_" + format(scrapeInt[sI-1] ) + "to" + format(scrapeInt[sI] ) + "recipes.p")
    print(filename)
    #pickle.dump(scrapedInfo, open( "scrapedInfo_BBCgdfd_100recipes.p", "wb" ) )    
    #     pickle.dump(scrapedInfo, open( filename, "wb" ) ) 
    
    with open( filename, "wb" ) as f:
        pickle.dump(scrapedInfo,f,-1)

scrape_chunk_loop : 33
0
10
20
30
40
50
60
70
80
90
100
/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/bbc_goodfood_recipes_pickles/scrapedInfo_BBCgdfd_3259to3360recipes.p


In [ ]:
# test = pickle.load(open( filename, "rb" ) )